In [2]:
from mpasanalysis import *
from e3sm_simulations import *

In [ ]:
# get paths of restart files, monthly mean output files, processed climatology files and output figures
ts_yr0 = 21
ts_yr1 = 50
plt_yr0 = 21
plt_yr1 = 50
rest_yr = 51
nmon = 12 # 12 for production and 1 for testing
run = E3SMSimulation(database='e3sm_simulations.xml', runname='gl-mesh-gm1800')
data_root = run.get_path(comp='ocn', climo_yr0=ts_yr0, climo_yr1=ts_yr1,
                         ts_yr0=ts_yr0, ts_yr1=ts_yr1, rest_yr=rest_yr)
rst_root = data_root['rst_root']
mon_root = data_root['mon_root']
fig_root = data_root['fig_root']
rst_file = rst_root+'/mpaso.rst.{:04d}-01-01_00000.nc'.format(rest_yr)

# load mesh
mpasmesh = MPASMesh(filepath=rst_file)

# transects
transect_list = ['Nares Strait',
                 'Parry Channel',
                 'Jones Sound']

# transect
for transect_name in transect_list:
    trnsct = VerticalTransect(transect_name)
    path0 = mpasmesh.get_shortest_path(trnsct.lon0, trnsct.lat0, trnsct.lon1, trnsct.lat1, npoint_ref=2)

    nv = len(path0.idx_vertex)
    transport = np.zeros([plt_yr1-plt_yr0+1, nmon, nv])
    for i, y in enumerate(np.arange(plt_yr1-plt_yr0+1)+plt_yr0):
        print('{:04d}'.format(y))
        for j, m in enumerate(np.arange(nmon)+1):
#             print('{:04d}-{:02d}'.format(y, m))
            mon_file = mon_root+'/mpaso.hist.am.timeSeriesStatsMonthly.{:04d}-{:02d}-01.nc'.format(y, m)
#             print(mon_file)
            mpasdata = MPASOData(filepath=mon_file, filepath_mesh=rst_file)
            transport0, dist0 = mpasdata.get_transport_cumulative(path=path0, varname_prefix='timeMonthly_avg_',
                                                                  bolus=True)
            transport[i,j,:] = transport0[0]
    # dist = dist0
    dist = gc_distance(np.array(path0.lon_vertex), np.array(path0.lat_vertex), trnsct.lon0, trnsct.lat0)

    # save data
    path_transport = fig_root+'/data_transport_LabSea_'+transect_name.replace(" ", "-")+'_volume.npz'
    np.savez(path_transport, transport=transport, distance=dist)
    

Pre-defined transect 'Nares Strait'.
Vertex closest to P0: 284.12319 77.95565
Vertex closest to P1: 287.59652 77.91273
 - Vertex closest to RefP1: 285.35058 77.85600
 - Vertex closest to RefP2: 286.38835 78.02253
0021
0022
0023
0024
0025
0026
0027
0028
0029
0030
0031
